In [4]:
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
sco_exfa = client['SCIENCEON']['ExpertFactor']
sco_aut = client['SCIENCEON']['Author']

ntis_exfa = client['NTIS']['ExpertFactor']
ntis_aut = client['NTIS']['Author']
ntis_raw = client['NTIS']['Rawdata']

dbpia_exfa = client['DBPIA']['ExpertFactor']
dbpia_aut = client['DBPIA']['Author']

id_domestic = client['ID']['Domestic']

keyid = 588 #input

sco_key_query = sco_exfa.find({ 'keyId' : keyid })
ntis_key_query = ntis_exfa.find({ 'keyId' : keyid })
dbpia_key_query = dbpia_exfa.find({ 'keyId' : keyid })

key_querys = [sco_key_query, ntis_key_query, dbpia_key_query]
#key_querys = [ntis_key_query]
auts = [sco_aut, ntis_aut, dbpia_aut]
site = ['Scienceon', 'NTIS', 'DBPIA']

a_id = []
all_name_inst = []
all_name = []
all_inst = []
all_site = []
reCopy = []
reCopy_site = []
reCopy_aid = []
Answer_list = []
Answer_dict = {}

for i in range(len(key_querys)):
    #print(key_querys[i])
    for key_query in key_querys[i]:
        a_id.append(key_query['A_ID'])

        if site[i] == 'NTIS' : 
            # if "11638621" == a_id[-1] :
            #print("check")
            ntis_raw_query = ntis_raw.find_one({'$and':[{'keyId':keyid},{'mngId':a_id[-1]}]})
            
            if ntis_raw_query == None :
            #print("check2")
                continue
    
        aut_query = auts[i].find_one({'_id':key_query['A_ID']})
        #print(aut_query)
        #all_name_inst.append(aut_query['name'] + '/' + aut_query['inst'])
        all_name.append(aut_query['name'])
        all_inst.append(aut_query['inst'].replace("(주) ", "").replace("(주)", "").split(' ')[0])
        all_site.append(site[i])


        #if all_name[-1] == '송재오' :
        #    print(aut_query['inst'])
        Answer = {'name' : all_name[-1], site[i] : all_inst[-1],  }

   
        if all_name[-1] not in Answer_dict and all_name[-1]+'0' not in Answer_dict :
            Answer_dict[all_name[-1]] = Answer
         #   print("insert")
        else :
            
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = all_name[-1]

              
                if tempName in Answer_dict :        # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = all_name[-1]+str(count)  # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                       # print(tempName)
                        break
                    temp = Answer_dict[tempName]

                
                      
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' : 
                        src = ""
                        tgt = ""

                        if len(all_inst[-1]) >= len(temp[key]):
                            src = temp[key]
                            tgt = all_inst[-1]

                        elif len(all_inst[-1]) < len(temp[key]):
                            src = all_inst[-1]
                            tgt = temp[key]
                        # if tempName == '김영미' :
                        #     print(all_inst[-1], temp[key])
                        #     print(ssrc, ttgt)


                        if key == site[i] :# 사이트가 동일할때
                            if temp[key] == all_inst[-1] or (src != "" and src in tgt) :  # 소속 같을때
                                flag = False
                                break
                            elif all_name[-1]+str(count+1) not in Answer_dict :
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                #if tempName != all_name[-1] :
                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    del Answer_dict[all_name[-1]]

                        else :# 사이트가 다를때 
                            if temp[key] == all_inst[-1] or (src != "" and src in tgt):  # 소속 같을때
                                #if len(src) == 0:
                                 #   flag = False
                                #else:
                                    Answer_dict[tempName][site[i]] = all_inst[-1]
                                    flag = False
                                    break
                            
                            elif all_name[-1]+str(count+1) not in Answer_dict :
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                #if tempName != all_name[-1] :
                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    del Answer_dict[all_name[-1]]   

                count += 1


        # Answer_list.append(Answer)

#Answer_dict.sort()
#print(len(Answer_dict))
#print(Answer_dict)
print(sorted(Answer_dict.items()))


{'_id': ObjectId('61939447e9b04a9d64abe067'), 'qryKeyword': ['빅데이터', '소셜네트워크'], 'qryTime': '202111162021', 'keyId': 588, 'loop': 13, 'issue_inst': '한국디지털정책학회', 'journal': '디지털융복합연구 = Journal of digital convergence', 'issue_year': '2020', 'id': 'JAKO202008540580214', 'title': 'K-POP 아이돌 그룹 신한복 스타일에 대한 글로벌 반응: 블랙핑크 패션 사례', 'english_title': "The global response to K-POP idol group's New Hanbok: The case of Black Pink Fashion", 'abstract': "본 연구는 K-POP 가수의 신한복 스타일에 대한 국내&#183;외 소비자들의 반응을 알아보는 것을 목적으로 한다. '블랙핑크 한복'을 키워드로 포함하는 유튜브 영상과 사용자 댓글을 수집했으며, 소셜 네트워크 분석과 감성분석을 적용했다. 첫째, 블랙핑크의 신한복 스타일은 안무하기 편하도록 미니드레스 형태로 제작되었으며, 전통적 요소를 더해 현대적으로 재해석한 것으로 나타났다. 둘째, 블랙핑크의 신한복 패션에 대해 국내 반응에서는 노출에 대한 이슈가 논란이 된 키워드로 등장한 반면, 해외 반응에서는 나타나지 않았다. 셋째, 감성분석 결과 국내보다 해외 반응에서 신한복 스타일에 대해 더 긍정적인 평가를 보였다. 본 연구는 광범위한 소비자들의 반응을 알아보고 신한복에 대한 긍부정 요소를 파악하여, 신한복이 나아가야 할 방향을 제시했다는데 의의가 있다.", 'english_abstract': '', 'citation': 0, 'author': '최영현;천탠이;이규혜;', 'author_inst': '한양대학교 의류학과;한양대학교 의류학과;한양대학교 의류학과 휴먼테크융합전공;', 'start_p

In [42]:
import json

answer = None
numCor = 0
numErr = 0
answerCounter = {}


def ansCheck(result, name) :
    global answer, numCor, numErr, answerCounter
    flag = True

    numK = len(result.keys())   
    if numK-1 != len(answer[name].keys()) : #site 개수 다르면 false
        flag = False
    else :
        for site in result.keys() :
            if site != 'name' : # site name이 다르면
                if site not in answer[name].keys() or result[site] not in answer[name][site] : #answer[name][site] != result[site] and answer[name][site] != result[site].split(' ')[0]): # result key와 answer key가 다르면
                    flag = False
                # elif answer[name][site] != result[site] and answer[name][site] != result[site].split(' ')[0]: 
                #     flag = False
                    # if numK > 2: # site가 2개 이상이면(name 포함 3개이상이면)
                    # if answer[name][site] != result[site] and answer[name][site] != result[site].split(' ')[0] :
                    #         flag = False
                    # else :
                    #     if answer[name][site] != result[site] and answer[name][site] != result[site].split(' ')[0]:
                    #         flag = False
    if flag :
        numCor += 1
        answerCounter[name] += 1
        #print(f"Correct \n - Answer : {answer[name]}\n - Result : {result}")
    else :
        print(f"Not Correct \n - Answer : {answer[name]}\n - Result : {result}")
        if len(answer[name].keys()) > 1 :
            numErr += 1
    return flag

with open('answer.json', 'r',encoding='UTF8') as a_json :


    answer = json.load(a_json)
    numAns = print(len(answer))
    
    for key in answer.keys() :
        answerCounter[key] = 0

    for answer_one in Answer_dict:
        test = answer_one
        #print(test)

        name = Answer_dict[test]['name']
        
        if name in answer :
            ansCheck(Answer_dict[test], name)

        else :
            count = 0
            while name+str(count) in answer :
                # if name+count in answer :
                print(f"Check {name+str(count)}")
                if ansCheck(Answer_dict[test], name+str(count)) :
                    break
                count += 1

    print(f"num코렉트 {numCor}")
    print(f"numEror {numErr}")

    tempCount = 0
    for key in answerCounter.keys() :
        if answerCounter[key] == 0 :
            tempCount += 1
            print(key)
            print(answer[key])
    print(tempCount)   

1802
Check 복경수0
Check 복경수0
Not Correct 
 - Answer : {'Scienceon': '원광대학교', 'DBPIA': '원광대학교'}
 - Result : {'name': '복경수', 'Scienceon': '충북대학교', 'DBPIA': '충북대학교'}
Check 복경수1
Check 이정하0
Not Correct 
 - Answer : {'Scienceon': '과학기술연합대학원대학교 건설환경공학과'}
 - Result : {'name': '이정하', 'Scienceon': '한국건설기술연구원', 'DBPIA': '한국건설기술연구원'}
Check 이정하1
Check 이정하0
Check 구철모0
Check 서주환0
Not Correct 
 - Answer : {'Scienceon': '경희대학교 예술.디자인대학 환경조경디자인학과'}
 - Result : {'name': '서주환', 'DBPIA': '한국과학기술정보연구원'}
Check 서주환1
Check 서주환0
Not Correct 
 - Answer : {'Scienceon': 'Department of IT Management, Kosin University', 'DBPIA': '고신대학교'}
 - Result : {'name': '반재훈', 'DBPIA': '고신대학교'}
Not Correct 
 - Answer : {'Scienceon': 'Department of IT Management, Kosin University', 'DBPIA': '고신대학교'}
 - Result : {'name': '반재훈', 'Scienceon': 'Department'}
Check 이정하0
Not Correct 
 - Answer : {'Scienceon': '과학기술연합대학원대학교 건설환경공학과'}
 - Result : {'name': '이정하', 'DBPIA': '세명대학교'}
Check 이정하1
Not Correct 
 - Answer : {'Scienceon': '한국건설기술연구원

In [43]:
print("(주) 제오시스 기업부설연구소".replace("(주) ",""))

제오시스 기업부설연구소
